In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [2]:
class getBlueprint:
    def getCannyEdge(image):
        try:
            grey = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            bilateralFilter = cv.bilateralFilter(grey, 11, 17, 17)
            # thresh = cv.threshold(bilateralFilter, 60, 255, cv.THRESH_BINARY)[1]
        except:
            bilateralFilter = image
        imageMedian = np.median(image)
        lowerThreshold = max(0, (0.7 * imageMedian))
        upperThreshold = min(255, (0.7 * imageMedian))
        CannyEdged = cv.Canny(bilateralFilter, lowerThreshold , upperThreshold)
        for i in range(CannyEdged.shape[0]):
            for j in range(CannyEdged.shape[1]):
                if(CannyEdged[i][j]):
                    CannyEdged[i][j] = 0
                else:
                    CannyEdged[i][j] = 255
        return CannyEdged

    def getStoreBoundries(image):
        area = image.shape[0]*image.shape[1]
        minarea = min(image.shape[0],image.shape[1]) 
        maxarea = area*(minarea-1)//minarea
        boundryImage = np.copy(image)
        for i in range(boundryImage.shape[0]):
            for j in range(boundryImage.shape[1]):
                if(not boundryImage[i][j]):
                    boundryImage[i][j] = 255
        
        (cnts, _) = cv.findContours(cannyImg, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        cnts=sorted(cnts, key = cv.contourArea, reverse = True)
        storeBoundries = []
        for c in cnts:
            x,y,w,h = cv.boundingRect(c)
            if(w > image.shape[1]//15 and h > image.shape[0]//15 and w*h in range(minarea,maxarea) ):
                storeBoundries.append([x,y,w,h])
                cv.rectangle(boundryImage,(x,y),(x+w,y+h),(0,0,255),1)
        return (boundryImage, storeBoundries)
    
    def getStoreNodes(image, storeBoundries):
        storeNodes = []
        i=0
        for [x,y,w,h] in storeBoundries:
            if(not([x+w//2, y+h//2] in storeNodes)):
                i+=1
                storeNodes.append([x+w//2, y+h//2])
            # cv.circle(image, (x+w//2,y+h//2), 2, (0, 0, 0), 2)
            cv.putText(image, str(i), (x + w // 2, y + h // 2), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)

        return (image, storeNodes)
            


In [3]:
image = cv.imread('./Blueprints/test.png')
cannyImg = getBlueprint.getCannyEdge(image)
cannyImg = getBlueprint.getCannyEdge(cannyImg)
# cv.imshow('cannyImg',cannyImg)
cv.imwrite('./Outputs/cannyImg.png',cannyImg)

(storeImg, storeBoundries) = getBlueprint.getStoreBoundries(np.copy(cannyImg))
# cv.imshow('Stores',storeImg)
cv.imwrite('./Outputs/Stores.png',storeImg)


(nodeImg, storeNodes) = getBlueprint.getStoreNodes(np.copy(storeImg), storeBoundries)
# cv.imshow('Nodes',nodeImg)
cv.imwrite('./Outputs/Nodes.png',nodeImg)


# print(cannyImg.shape,storeImg.shape)
# print(len(storeBoundries),len(storeNodes))



# cv.waitKey(0)

True